# Notebook for Creating High Wage Outputs with Racial Demographics Breakdown

#### This notebook is currently written to create high wage outputs for 2020.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import warnings
import os
import re
from jqi_functions import *
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format='retina'

## Set the desired year and the corresponding cost of living year

In [2]:
year = '2010'

## Creating IPUMS dataframe

#### IPUMS Data
`cleaned_ipums` is a function to generate a cleaned pandas dataframe using IPUMS data, filtering it down to California only and the desired year. By setting the `demo` parameter to True, the dataframe will also include the racial demographic information for each record in the dataset. The year needs to be entered in string format as a parameter.

In [3]:
ca_ipums = cleaned_ipums(year, demo=True)

#### Cost of living needs to be updated each year.

In [4]:
cost_of_living = pd.read_csv(f'data/cost_of_living/united-way-col-1A1PS1C{year}.csv')

### Create county lookup dataframe

Expanding the `county_info` dataframe to include cost of living metrics. This dataframe is used when industry information in a geographic area is too sparse and the next largest geographic area needs to be used instead.

In [5]:
county_info = pd.read_csv('data/county_to_regions_key.csv')

In [6]:
county_info = county_info[['County', 'COUNTYFIP', 'Rural/Urban', 'CERF Regions', 'Population']]

In [7]:
county_info = pd.merge(county_info, cost_of_living, left_on = 'CERF Regions', right_on = 'Regions')

In [8]:
county_info = county_info.rename(columns = {'Cost of Living':'Regional COL'})
county_info = county_info.drop(columns=['Regions'])

In [9]:
county_info['State COL'] = cost_of_living.iloc[13][1]

View of final `county_info` dataframe.

In [10]:
county_info.head()

,County,COUNTYFIP,Rural/Urban,CERF Regions,Population,Regional COL,State COL
0,Alameda,1,Urban,Bay Area,1656754,82928.33,66134.41
1,Contra Costa,13,Urban,Bay Area,1142251,82928.33,66134.41
2,Solano,95,Urban,Bay Area,441829,82928.33,66134.41
3,San Mateo,81,Urban,Bay Area,767423,82928.33,66134.41
4,Santa Clara,85,Urban,Bay Area,1927470,82928.33,66134.41


In [11]:
ca_ipums = pd.merge(ca_ipums, county_info, on = 'COUNTYFIP')

View of final `ca_ipums` dataframe.

In [12]:
ca_ipums.head()

,YEAR,COUNTYFIP,INDNAICS,PERWT,RACE,HISPAN,INCWAGE,NAICS Code,Industry Title,Industry,Crosswalk Value,County,Rural/Urban,CERF Regions,Population,Regional COL,State COL
0,2010,75,3341,76.0,1,0,195000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,8,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
1,2010,75,3341,76.0,1,0,195000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,9,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
2,2010,75,3341,58.0,4,0,25000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,8,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
3,2010,75,3341,58.0,4,0,25000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,9,San Francisco,Urban,Bay Area,874961,82928.33,66134.41
4,2010,75,3341,54.0,8,0,140000,3341,computer and peripheral equipment manufacturing,computer and peripheral equipment manufacturing,8,San Francisco,Urban,Bay Area,874961,82928.33,66134.41


## Create EDD Dataframe

#### EDD Data
The year for EDD data must be specified.

These CSV files are filtered and cleaned versions of the raw EDD Current Employment Statistics dataset. These CSV files can be created for upcoming years with the notebook `multiyear-edd-data-creation.ipynb`

In [13]:
edd = pd.read_csv(f'data/edd/edd_{year}.csv')

View of final `edd` dataframe.

In [14]:
edd.head()

,Area Type,Area Name,Year,Month,Date,Series Code,Seasonally Adjusted,Current Employment,Industry Title,COUNTYFIP,County,Rural/Urban,CERF Regions,Crosswalk Value
0,County,Alameda,2010,January,01/01/2010,80000000,N,22300,other services,1,Alameda,Urban,Bay Area,32
1,County,Alameda,2010,February,02/01/2010,80000000,N,22500,other services,1,Alameda,Urban,Bay Area,32
2,County,Alameda,2010,March,03/01/2010,80000000,N,22900,other services,1,Alameda,Urban,Bay Area,32
3,County,Alameda,2010,April,04/01/2010,80000000,N,23200,other services,1,Alameda,Urban,Bay Area,32
4,County,Alameda,2010,May,05/01/2010,80000000,N,23400,other services,1,Alameda,Urban,Bay Area,32


## Breakdown dataframes by race

In [15]:
ca_ipums_latino = ca_ipums.loc[ca_ipums['HISPAN'] != 0]
ca_ipums_no_latino = ca_ipums.loc[ca_ipums['HISPAN'] == 0]
ca_ipums_white = ca_ipums_no_latino.loc[ca_ipums_no_latino['RACE'] == 1]
ca_ipums_black = ca_ipums_no_latino.loc[ca_ipums_no_latino['RACE'] == 2]
ca_ipums_native = ca_ipums_no_latino.loc[ca_ipums_no_latino['RACE'] == 3]
ca_ipums_asian = ca_ipums_no_latino.loc[(ca_ipums_no_latino['RACE'] == 4) |
                                   (ca_ipums_no_latino['RACE'] == 5) |
                                   (ca_ipums_no_latino['RACE'] == 6)]
ca_ipums_other = ca_ipums_no_latino.loc[ca_ipums_no_latino['RACE'] == 7]
ca_ipums_multi = ca_ipums_no_latino.loc[(ca_ipums_no_latino['RACE'] == 8) |
                                   (ca_ipums_no_latino['RACE'] == 9)]

In [16]:
wt_counts = {}
for code in ca_ipums['Crosswalk Value'].unique():
    code_copy = ca_ipums.loc[ca_ipums['Crosswalk Value'] == code].copy()
    wt_counts[code] = code_copy['PERWT'].sum()

In [17]:
def append_race_ratio(df, wt_counts):
    df['race_ratio'] = 0
    for code in df['Crosswalk Value'].unique():
        perwt = df['PERWT'][df['Crosswalk Value'] == code].sum()
        df['race_ratio'][df['Crosswalk Value'] == code] = perwt / wt_counts[code]
    return df

In [18]:
ca_ipums_white = append_race_ratio(ca_ipums_white, wt_counts)

In [19]:
ca_ipums_latino = append_race_ratio(ca_ipums_latino, wt_counts)

In [20]:
ca_ipums_black = append_race_ratio(ca_ipums_black, wt_counts)

In [21]:
ca_ipums_native = append_race_ratio(ca_ipums_native, wt_counts)

In [22]:
ca_ipums_asian = append_race_ratio(ca_ipums_asian, wt_counts)

In [23]:
ca_ipums_other = append_race_ratio(ca_ipums_other, wt_counts)

In [24]:
ca_ipums_multi = append_race_ratio(ca_ipums_multi, wt_counts)

## Add High Wage Features

`add_geo_high_wages` is a function that adds the following engineered features:
- Above Threshold (Number of records above respective cost of living threshold)
- Weighted above threshold (Above Threshold multiplied by person weight variable)
- Unweighted industry counts (Number of records in that industry)
- Weighted industry counts (Sum of person weight values in that industry)
- Weighted high wage percentage (Weighted Above Threshold divided by Weighted Industry Counts as a percentage)

The features are created for the following geographical levels:
- Region
- California

In [25]:
ca_ipums_hw_white = add_geo_high_wages(ca_ipums_white)
ca_ipums_hw_latino = add_geo_high_wages(ca_ipums_latino)
ca_ipums_hw_black = add_geo_high_wages(ca_ipums_black)
ca_ipums_hw_native = add_geo_high_wages(ca_ipums_native)
ca_ipums_hw_asian = add_geo_high_wages(ca_ipums_asian)
ca_ipums_hw_other = add_geo_high_wages(ca_ipums_other)
ca_ipums_hw_multi = add_geo_high_wages(ca_ipums_multi)

## Create High Wage Outputs Dataframe

`edd_to_hw` is the function that outputs the values needed to create the high wage output dataframe. This portion of the notebook runs through every unique combination of region, industry, and date, to get that respective output and add it to the dataframe.

Because of the nested structure of the EDD industries, only a small selection of EDD industries can be used to ensure that individuals in nested industries are not counted twice. The selection of these industries different per region, so the series code of each industry is documented in the `region_series_codes` global variable in the `jqi_functions.py` library. Each of these codes were then assigned their own crosswalk value, which align with each crosswalk value assigned to each IPUMS industry. For generating high wage outputs, we only iterate through the EDD industries that have been selected and have a designated crosswalk value.

Getting unique values for each region, industry, and date.

In [26]:
regions_ipums = ca_ipums['CERF Regions'].unique()

In [27]:
crosswalk_vals = sorted(edd['Crosswalk Value'].unique())

In [28]:
dates_edd = edd['Date'].unique()

In [29]:
total_iterations = len(regions_ipums) * len(crosswalk_vals) * len(dates_edd)

#### White

In [30]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [31]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw_white, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [32]:
df_dict_white = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output_white = pd.DataFrame(df_dict_white)
hw_output_white = hw_output_white[hw_output_white['Industry'].notna()]
hw_output_white['Date']= pd.to_datetime(hw_output_white['Date'])
hw_output_white['High Wage Count'] = hw_output_white['High Wage Count'].astype(int)
hw_output_white = hw_output_white.sort_values(by=['Industry', 'Region', 'Date'])
hw_output_white = pd.merge(hw_output_white, cost_of_living, left_on='Region', right_on='Regions')
hw_output_white = hw_output_white[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output_white = hw_output_white.drop_duplicates()
hw_output_white['Output Race'] = 'White'

View of final `hw_output_white` dataframe.

In [33]:
hw_output_white.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,16086,105200.0,53849.84,White
1,accommodation and food services,2010-02-01,Inland Empire,16162,105700.0,53849.84,White
2,accommodation and food services,2010-03-01,Inland Empire,16315,106700.0,53849.84,White
3,accommodation and food services,2010-04-01,Inland Empire,16575,108400.0,53849.84,White
4,accommodation and food services,2010-05-01,Inland Empire,16575,108400.0,53849.84,White


#### Latino

In [34]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [35]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw_latino, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [36]:
df_dict_latino = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output_latino = pd.DataFrame(df_dict_latino)
hw_output_latino = hw_output_latino[hw_output_latino['Industry'].notna()]
hw_output_latino['Date']= pd.to_datetime(hw_output_latino['Date'])
hw_output_latino['High Wage Count'] = hw_output_latino['High Wage Count'].astype(int)
hw_output_latino = hw_output_latino.sort_values(by=['Industry', 'Region', 'Date'])
hw_output_latino = pd.merge(hw_output_latino, cost_of_living, left_on='Region', right_on='Regions')
hw_output_latino = hw_output_latino[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output_latino = hw_output_latino.drop_duplicates()
hw_output_latino['Output Race'] = 'Latino'

View of final `hw_output_latino` dataframe.

In [37]:
hw_output_latino.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,5744,105200.0,53849.84,Latino
1,accommodation and food services,2010-02-01,Inland Empire,5772,105700.0,53849.84,Latino
2,accommodation and food services,2010-03-01,Inland Empire,5826,106700.0,53849.84,Latino
3,accommodation and food services,2010-04-01,Inland Empire,5919,108400.0,53849.84,Latino
4,accommodation and food services,2010-05-01,Inland Empire,5919,108400.0,53849.84,Latino


#### Black

In [38]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [39]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw_black, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [40]:
df_dict_black = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output_black = pd.DataFrame(df_dict_black)
hw_output_black = hw_output_black[hw_output_black['Industry'].notna()]
hw_output_black['Date']= pd.to_datetime(hw_output_black['Date'])
hw_output_black['High Wage Count'] = hw_output_black['High Wage Count'].astype(int)
hw_output_black = hw_output_black.sort_values(by=['Industry', 'Region', 'Date'])
hw_output_black = pd.merge(hw_output_black, cost_of_living, left_on='Region', right_on='Regions')
hw_output_black = hw_output_black[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output_black = hw_output_black.drop_duplicates()
hw_output_black['Output Race'] = 'Black'

View of final `hw_output_black` dataframe.

In [41]:
hw_output_black.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,10180,105200.0,53849.84,Black
1,accommodation and food services,2010-02-01,Inland Empire,10229,105700.0,53849.84,Black
2,accommodation and food services,2010-03-01,Inland Empire,10325,106700.0,53849.84,Black
3,accommodation and food services,2010-04-01,Inland Empire,10490,108400.0,53849.84,Black
4,accommodation and food services,2010-05-01,Inland Empire,10490,108400.0,53849.84,Black


#### American Indian/Alaska Native

In [42]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [43]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw_native, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [44]:
df_dict_native = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output_native = pd.DataFrame(df_dict_native)
hw_output_native = hw_output_native[hw_output_native['Industry'].notna()]
hw_output_native['Date']= pd.to_datetime(hw_output_native['Date'])
hw_output_native['High Wage Count'] = hw_output_native['High Wage Count'].astype(int)
hw_output_native = hw_output_native.sort_values(by=['Industry', 'Region', 'Date'])
hw_output_native = pd.merge(hw_output_native, cost_of_living, left_on='Region', right_on='Regions')
hw_output_native = hw_output_native[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output_native = hw_output_native.drop_duplicates()
hw_output_native['Output Race'] = 'American Indian/Alaska Native'

View of final `hw_output_native` dataframe.

In [45]:
hw_output_native.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,0,48100.0,53849.84,American Indian/Alaska Native
1,accommodation and food services,2010-02-01,Inland Empire,0,48000.0,53849.84,American Indian/Alaska Native
2,accommodation and food services,2010-03-01,Inland Empire,0,48500.0,53849.84,American Indian/Alaska Native
3,accommodation and food services,2010-04-01,Inland Empire,0,49000.0,53849.84,American Indian/Alaska Native
4,accommodation and food services,2010-05-01,Inland Empire,0,49200.0,53849.84,American Indian/Alaska Native


#### Asian

In [46]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [47]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw_asian, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [48]:
df_dict_asian = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output_asian = pd.DataFrame(df_dict_asian)
hw_output_asian = hw_output_asian[hw_output_asian['Industry'].notna()]
hw_output_asian['Date']= pd.to_datetime(hw_output_asian['Date'])
hw_output_asian['High Wage Count'] = hw_output_asian['High Wage Count'].astype(int)
hw_output_asian = hw_output_asian.sort_values(by=['Industry', 'Region', 'Date'])
hw_output_asian = pd.merge(hw_output_asian, cost_of_living, left_on='Region', right_on='Regions')
hw_output_asian = hw_output_asian[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output_asian = hw_output_asian.drop_duplicates()
hw_output_asian['Output Race'] = 'Asian'

View of final `hw_output_asian` dataframe.

In [49]:
hw_output_asian.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,18641,105200.0,53849.84,Asian
1,accommodation and food services,2010-02-01,Inland Empire,18730,105700.0,53849.84,Asian
2,accommodation and food services,2010-03-01,Inland Empire,18907,106700.0,53849.84,Asian
3,accommodation and food services,2010-04-01,Inland Empire,19208,108400.0,53849.84,Asian
4,accommodation and food services,2010-05-01,Inland Empire,19208,108400.0,53849.84,Asian


#### Some other race

In [50]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [51]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw_other, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [52]:
df_dict_other = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output_other = pd.DataFrame(df_dict_other)
hw_output_other = hw_output_other[hw_output_other['Industry'].notna()]
hw_output_other['Date']= pd.to_datetime(hw_output_other['Date'])
hw_output_other['High Wage Count'] = hw_output_other['High Wage Count'].astype(int)
hw_output_other = hw_output_other.sort_values(by=['Industry', 'Region', 'Date'])
hw_output_other = pd.merge(hw_output_other, cost_of_living, left_on='Region', right_on='Regions')
hw_output_other = hw_output_other[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output_other = hw_output_other.drop_duplicates()
hw_output_other['Output Race'] = 'Some other race'

View of final `hw_output_other` dataframe.

In [53]:
hw_output_other.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,-571,57100.0,53849.84,Some other race
1,accommodation and food services,2010-02-01,Inland Empire,-577,57700.0,53849.84,Some other race
2,accommodation and food services,2010-03-01,Inland Empire,-582,58200.0,53849.84,Some other race
3,accommodation and food services,2010-04-01,Inland Empire,-594,59400.0,53849.84,Some other race
4,accommodation and food services,2010-05-01,Inland Empire,-592,59200.0,53849.84,Some other race


#### Multiracial

In [54]:
industries = []
dates = []
regions = []
counts = []
emp_counts = []

For loop to populate lists for the high wage outputs. This will take some time to finish running.

In [55]:
progress_count = 0
for region in regions_ipums:
    for code in crosswalk_vals:
        for date in dates_edd:
            hw_count, hw_perc, employment_count, industry = edd_to_hw(edd, ca_ipums_hw_multi, region, code, date, 10)
            industries.append(industry)
            dates.append(date)
            regions.append(region)
            counts.append(hw_count)
            emp_counts.append(employment_count)
            progress_count += 1
            if progress_count % (total_iterations / 10) == 0:
                percent_done = int((progress_count / total_iterations) * 100)
                print(f'Progress: {percent_done}% Complete')

Progress: 10% Complete
Progress: 20% Complete
Progress: 30% Complete
Progress: 40% Complete
Progress: 50% Complete
Progress: 60% Complete
Progress: 70% Complete
Progress: 80% Complete
Progress: 90% Complete
Progress: 100% Complete


Creating a cleaned dataframe from the output lists.

In [56]:
df_dict_multi = {'Industry':industries, 'Date':dates, 'Region':regions, 'High Wage Count':counts, 'Employment Count':emp_counts}
hw_output_multi = pd.DataFrame(df_dict_multi)
hw_output_multi = hw_output_multi[hw_output_multi['Industry'].notna()]
hw_output_multi['Date']= pd.to_datetime(hw_output_multi['Date'])
hw_output_multi['High Wage Count'] = hw_output_multi['High Wage Count'].astype(int)
hw_output_multi = hw_output_multi.sort_values(by=['Industry', 'Region', 'Date'])
hw_output_multi = pd.merge(hw_output_multi, cost_of_living, left_on='Region', right_on='Regions')
hw_output_multi = hw_output_multi[['Industry', 'Date', 'Region', 'High Wage Count', 'Employment Count', 'Cost of Living']]
hw_output_multi = hw_output_multi.drop_duplicates()
hw_output_multi['Output Race'] = 'Multiracial'

View of final `hw_output_multi` dataframe.

In [57]:
hw_output_multi.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,5488,105200.0,53849.84,Multiracial
1,accommodation and food services,2010-02-01,Inland Empire,5514,105700.0,53849.84,Multiracial
2,accommodation and food services,2010-03-01,Inland Empire,5566,106700.0,53849.84,Multiracial
3,accommodation and food services,2010-04-01,Inland Empire,5655,108400.0,53849.84,Multiracial
4,accommodation and food services,2010-05-01,Inland Empire,5655,108400.0,53849.84,Multiracial


### Concatenate all dataframes and export

Code to export the dataframe as a CSV file - change file path if needed and uncomment to run.

In [58]:
hw_output_demo = pd.concat([hw_output_white, hw_output_black, hw_output_latino, hw_output_asian, hw_output_native, hw_output_other, hw_output_multi], ignore_index=True)

In [59]:
hw_output_demo.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race
0,accommodation and food services,2010-01-01,Inland Empire,16086,105200.0,53849.84,White
1,accommodation and food services,2010-02-01,Inland Empire,16162,105700.0,53849.84,White
2,accommodation and food services,2010-03-01,Inland Empire,16315,106700.0,53849.84,White
3,accommodation and food services,2010-04-01,Inland Empire,16575,108400.0,53849.84,White
4,accommodation and food services,2010-05-01,Inland Empire,16575,108400.0,53849.84,White


#### Add Region Population to Dataframe

In [60]:
reg_pop = county_info.groupby(by='CERF Regions').sum()[['Population']].reset_index()
reg_pop

,CERF Regions,Population
0,Bay Area,7710026
1,Central Coast,2342005
2,Central San Joaquin,1752543
3,Eastern Sierra,188734
4,Inland Empire,4560470
5,Kern,887641
6,Los Angeles,10081570
7,North State,713754
8,Northern San Joaquin,1557179
9,Orange,3168044


In [61]:
hw_output_demo = pd.merge(hw_output_demo, reg_pop, left_on='Region', right_on='CERF Regions')
hw_output_demo = hw_output_demo.drop(columns=['CERF Regions'])
hw_output_demo = hw_output_demo.rename(columns={"Population": "Region Population"})
hw_output_demo.head()

,Industry,Date,Region,High Wage Count,Employment Count,Cost of Living,Output Race,Region Population
0,accommodation and food services,2010-01-01,Inland Empire,16086,105200.0,53849.84,White,4560470
1,accommodation and food services,2010-02-01,Inland Empire,16162,105700.0,53849.84,White,4560470
2,accommodation and food services,2010-03-01,Inland Empire,16315,106700.0,53849.84,White,4560470
3,accommodation and food services,2010-04-01,Inland Empire,16575,108400.0,53849.84,White,4560470
4,accommodation and food services,2010-05-01,Inland Empire,16575,108400.0,53849.84,White,4560470


In [62]:
hw_output_demo.to_csv(f'data/outputs/hw_outputs_w_race_{year}.csv', encoding='utf-8', index=False)

## Code for concatenating multiple years (2010-2020)

In [67]:
# hw_2010 = pd.read_csv('data/outputs/hw_outputs_w_race_2010.csv')
# hw_2011 = pd.read_csv('data/outputs/hw_outputs_w_race_2011.csv')
# hw_2012 = pd.read_csv('data/outputs/hw_outputs_w_race_2012.csv')
# hw_2013 = pd.read_csv('data/outputs/hw_outputs_w_race_2013.csv')
# hw_2014 = pd.read_csv('data/outputs/hw_outputs_w_race_2014.csv')
# hw_2015 = pd.read_csv('data/outputs/hw_outputs_w_race_2015.csv')
# hw_2016 = pd.read_csv('data/outputs/hw_outputs_w_race_2016.csv')
# hw_2017 = pd.read_csv('data/outputs/hw_outputs_w_race_2017.csv')
# hw_2018 = pd.read_csv('data/outputs/hw_outputs_w_race_2018.csv')
# hw_2019 = pd.read_csv('data/outputs/hw_outputs_w_race_2019.csv')
# hw_2020 = pd.read_csv('data/outputs/hw_outputs_w_race_2020.csv')

In [68]:
# hw_2010['Year'] = 2010
# hw_2011['Year'] = 2011
# hw_2012['Year'] = 2012
# hw_2013['Year'] = 2013
# hw_2014['Year'] = 2014
# hw_2015['Year'] = 2015
# hw_2016['Year'] = 2016
# hw_2017['Year'] = 2017
# hw_2018['Year'] = 2018
# hw_2019['Year'] = 2019
# hw_2020['Year'] = 2020

In [69]:
# hw_output_concat = pd.concat([hw_2010, hw_2011, hw_2012, hw_2013, 
#                               hw_2014, hw_2015, hw_2016, hw_2017, 
#                               hw_2018, hw_2019, hw_2020])

In [70]:
# hw_output_concat.to_csv('data/outputs/hw_outputs_w_race_multiyear.csv', encoding='utf-8', index=False)